In [12]:
import pandas as pd
import time 
import csv

In [13]:
start  =  time.time()
def order_book_tranform(year,month,day,path,best_price_number):
    ## read file
    def read_file(year,month,day,path):
        data = []
        if len(str(month)) == 1:
            month_ = '0'+str(month)
        else:
            month_ = str(month)
        if len(str(day)) == 1:
            day_ = '0'+str(day)
        else:
            day_ = str(day)
        datapath = str(path) + str(year)+'.'+str(month_)+'.'+str(day_)+'.csv'
        data = pd.read_csv(datapath)
        data_CN14 = data[data.Series == data['Series'][0]]
        data = data_CN14
        return data

    def insert(order_book_data,data_to_insert,ob_position):
        top = order_book_data[0:ob_position]
        bottom = order_book_data[ob_position:]
        return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)

    def draw_out(order_book_data,ob_position):
        top = order_book_data[0:ob_position]
        bottom = order_book_data[ob_position+1:]
        return pd.concat((top,bottom)).reset_index(drop = True)

    def order_book_to_csv(order_book_bid,order_book_ask,data,i):
        order_book_bid_sum = order_book_bid[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
        order_book_ask_sum = order_book_ask[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
        order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)    
        #print order_book_bid_ask
        with open('order_book_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv','a') as f:
            order_book = csv.writer(f)
            order_book.writerow(["TimeStamp",data.TimeStamp[i-1:i].iloc[0]])
            order_book = csv.writer(f,delimiter=',')
            for i in range(0,min(len(order_book_bid_ask),best_price_number),1):#len(order_book_bid_ask)):
                order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])
        return order_book_bid_sum,order_book_ask_sum

    data = read_file(year,month,day,path)

    with open('order_book_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv', 'wb') as csvfile:
        f = csv.writer(csvfile) 

    data['QuantityDifference_'] = data['QuantityDifference']
    data_ask = data[(data.BidOrAsk == 'A')].reset_index(drop=True)
    data_bid = data[(data.BidOrAsk == 'B')].reset_index(drop=True)
    order_book_bid = []
    order_book_ask = []
    timestamp = data.TimeStamp[0]
    temp_ask = 0
    temp_bid = 0
    bid = []
    ask = []
    timestamp_ = []
    x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
    x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
    y = data[data['TimeStamp'].str.contains(timestamp)]
    bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # bid
    ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # ask
    a = bid[0].sort(['Price'],ascending = [False])
    b = ask[0].sort(['Price'],ascending = [True])    
    #order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
    #order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))
    order_book_bid = a[a.QuantityDifference != 0].reset_index(drop = True)
    order_book_ask = b[b.QuantityDifference != 0].reset_index(drop = True)
    #order_book_bid.append(0)
    #order_book_ask.append(0)
    order_book_bid_sum = order_book_bid[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
    order_book_ask_sum = order_book_ask[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
    order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)

    with open('order_book_'+str(year)+'_'+str(month)+'_'+str(day)+'.csv','a') as f:
        order_book = csv.writer(f)
        order_book.writerow(["Bid","Bid_Quantity","Ask","Ask_Quantity"])
        order_book.writerow(["TimeStamp",timestamp])
        order_book = csv.writer(f,delimiter=',')
        for i in range(0,min(len(order_book_bid_ask),best_price_number),1):
            order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])

    for i in range(len(y),len(data),1):
        #print i#,temp_bid,temp_ask
        time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                      int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                      int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
        if time_second > 57600:
            break
        if time_second == 32400 and time_second >= 57300:
            #order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False]).reset_index(drop = True)
            order_book_bid = order_book_bid.sort(['Price'],ascending = [False]).reset_index(drop = True)
            #order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True]).reset_index(drop = True)
            order_book_ask = order_book_ask.sort(['Price'],ascending = [True]).reset_index(drop = True)
            pass 

        if data.BidOrAsk[i] == 'A':
            data_ask_Quantity = data.BestQuantity[i]
            if int(data[['QuantityDifference']][i:i+1].values) > 0 :
                #if order_book_bid[temp_bid].Price[0] >= data[i:i+1].Price.iloc[0] and time_second < 32400:
                if order_book_bid.Price[0] >= data[i:i+1].Price.iloc[0] and time_second < 32400:
                    #for k in range(0,len(order_book_bid[temp_bid])):
                    for k in range(0,len(order_book_bid)):
                        #diff = order_book_bid[temp_bid].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                        diff = order_book_bid.QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]                    
                        #if order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff >= 0:
                        if order_book_bid.Price[k] >= data[i:i+1].Price.iloc[0] and diff >= 0:    
                            #order_book_bid[temp_bid].QuantityDifference_[k] = diff
                            order_book_bid.QuantityDifference_[k] = diff
                            data[i:i+1].QuantityDifference_.iloc[0] = 0
                            break
                        #elif order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff < 0:
                        elif order_book_bid.Price[k] >= data[i:i+1].Price.iloc[0] and diff < 0:    
                            #order_book_bid[temp_bid].QuantityDifference_[k] = 0
                            order_book_bid.QuantityDifference_[k] = 0
                            data[i:i+1].QuantityDifference_.iloc[0] = -diff
                            pass
                        else:
                            break
                if data.TimeStamp[i] == x1[temp_ask]:
                    #print 'timestamp = %s'%(data.TimeStamp[i])
                    #print order_book_ask
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    #order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                    order_book_ask = insert(order_book_ask,data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                    #length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                    if time_second > 32400 and time_second < 57300: 
                        #if position_ == 0 and len(order_book_ask[temp_ask]) > 1
                        if position_ == 0 and len(order_book_ask) > 1:   
                            if order_book_ask[position_ + 1:position_ + 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:  
                                print 'Some error1(Ask & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass
                        #elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                        elif 0 < position_< (len(order_book_ask)-1):
                            #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                print 'Some error1(Ask & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'   
                                pass
                        #elif position_ == len(order_book_ask[temp_ask])-1:
                        elif position_ == len(order_book_ask)-1:    
                            #if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:        
                                print 'Some error1(Ask & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'                
                                pass
                        #elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                        elif position_ == 0 and len(order_book_ask) == 1:
                            #print 'position and Best Price is True' 
                            pass
                    else:
                        #print 'before open market'
                        pass
                elif data.TimeStamp[i] != x1[temp_ask]:
                    if temp_ask == 0:
                        #print 'timestamp = %s'%(data.TimeStamp[i])
                        #print order_book_ask
                        temp_ask = temp_ask + 1
                        #order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                        #order_book_ask = order_book_ask
                        best_price = data[i:(i+1)]['BestPrice']
                        position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                        #order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  
                        order_book_ask = insert(order_book_ask,data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  

                        if time_second > 32400 and time_second < 57300:
                            if position_ == 0 and len(order_book_ask[0]) > 1:
                                #if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                                if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                                    print 'Some error2(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                    break
                                else:
                                    #print 'position and Best Price is True'                     
                                    pass
                            #elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            elif 0 < position_< len(order_book_ask)-1:
                                #if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                if order_book_ask[position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                    print 'Some error2(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                    break
                                else:
                                    #print 'position and Best Price is True'  
                                    pass
                            #elif position_ == len(order_book_ask[temp_ask])-1:  
                            elif position_ == len(order_book_ask)-1: 
                                #if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                if order_book_ask[position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                    print 'Some error2(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                    break
                                else:
                                    #print 'position and Best Price is True'
                                    pass
                            #elif position_ == 0 and len(order_book_ask[0]) == 1:
                            elif position_ == 0 and len(order_book_ask) == 1:
                                #print 'position and Best Price is True'
                                pass
                        else:
                            #print 'before open market'
                            pass
                    else:
                        #print 'timestamp = %s'%(data.TimeStamp[i])
                        #print order_book_ask
                        #order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                        order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid,order_book_ask,data,i)
                        if round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) > 0.03 or\
                        round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) < 0:
                            if data[i-1:i].BidOrAsk.iloc[0] == 'A':
                                if order_book_ask_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                    #print 'Best ask quantity is ture'
                                    pass
                                else:
                                    print 'Best ask quantity is false'
                                    break 
                            else:
                                j = i - 1
                                while j >= 1:
                                    if data[j-1:j].BidOrAsk.iloc[0] == 'A':
                                        if order_book_ask_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                            #print 'Best ask quantity is ture'
                                            break
                                    else:
                                        j = j - 1
                                        pass
                        else:
                            pass
                        position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                        #order_book_ask.append(order_book_ask[temp_ask])
                        temp_ask = temp_ask + 1
                        #order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                        order_book_ask = insert(order_book_ask,data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                        if time_second > 32400 and time_second < 57300:
                            if position_ == 0:
                                #if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                    print 'Some error3(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                    break
                                else:
                                    #print 'position and Best Price is True'                     
                                    pass
                            #elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            elif 0 < position_< len(order_book_ask)-1:
                                #if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                                    print 'Some error3(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                    break
                                else:
                                    #print 'position and Best Price is True'  
                                    pass
                            #elif position_ == len(order_book_ask[temp_ask])-1: 
                            elif position_ == len(order_book_ask)-1: 
                                #if order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                if order_book_ask[position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                    print 'Some error3(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                    break
                                else:
                                    #print 'position and Best Price is True'                        
                                    pass
                            elif position_ == 0 and len(order_book_ask[0]) == 1:
                                #print 'position and Best Price is True'
                                pass
                        else:
                            #print 'before open market'
                            pass
            elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
                if data.TimeStamp[i] == x1[temp_ask]: 
                    #print 'timestamp = %s'%(data.TimeStamp[i])
                    #print order_book_ask
                    order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                    #position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                    position_ = order_book_ask[order_book_ask.OrderNumber == order_number_].index[0]                
                    price_ = data[i:i+1]['Price'].iloc[0]
                    if time_second > 32400 and time_second < 57300:
                        #if position_ == 0 and len(order_book_ask[temp_ask]) > 1:
                        if position_ == 0 and len(order_book_ask) > 1:
                            #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:    
                                print 'Some error4(Ask & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass       
                        #elif 0 < position_< len(order_book_ask[temp_ask])-1:
                        elif 0 < position_< len(order_book_ask)-1:
                            #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:                             
                                print 'Some error4(Ask & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'  
                                pass
                        #elif position_ == len(order_book_ask[temp_ask])-1:  
                        elif position_ == len(order_book_ask)-1:
                            #if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            if position_ > 0 and order_book_ask[position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                print 'Some error4(Ask & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])   
                                break
                            elif position_ == 0:
                                #print 'position and Best Price is True'
                                pass                        
                            else:
                                #print 'position and Best Price is True'                        
                                pass
                        #elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                        elif position_ == 0 and len(order_book_ask) == 1:    
                            #print 'position and Best Price is True'
                            pass
                    else:
                        #print 'before open market' 
                        pass
                    #if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    if order_book_ask[(order_book_ask.OrderNumber == order_number_)&(order_book_ask.Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):        
                        #order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                        order_book_ask = order_book_ask.drop(order_book_ask.index[[position_]]).reset_index(drop = True)                
                    else:
                        #order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                        order_book_ask['QuantityDifference'][order_book_ask.OrderNumber == order_number_] = order_book_ask['QuantityDifference'][order_book_ask.OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]

                elif data.TimeStamp[i] != x1[temp_ask]:
                    #print 'timestamp = %s'%(data.TimeStamp[i])
                    #print order_book_ask
                    order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid,order_book_ask,data,i)
                    if round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) > 0.03 or\
                    round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) < 0:
                        if data[i-1:i].BidOrAsk.iloc[0] == 'A':
                            if order_book_ask_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                #print 'Best ask quantity is ture'
                                pass
                            else:
                                print 'Best ask quantity is false'
                                break 
                        else:
                            j = i - 1
                            while j >= 1:
                                #print j
                                #print data[j-1:j].BidOrAsk.iloc[0]
                                if data[j-1:j].BidOrAsk.iloc[0] == 'A':
                                    if order_book_ask_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                        #print 'Best ask quantity is ture'
                                        break
                                else:
                                    j = j - 1
                                    pass
                    #order_book_ask.append(order_book_ask[temp_ask])
                    order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                    #position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                    position_ = order_book_ask[order_book_ask.OrderNumber == order_number_].index[0]
                    price_ = data[i:i+1]['Price'].iloc[0]
                    temp_ask = temp_ask + 1
                    if time_second > 32400 and time_second < 57300:
                        #if position_ == 0 and len(order_book_ask[temp_ask]) > 1:
                        if position_ == 0 and len(order_book_ask) > 1:
                            #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                                print 'Some error5(Ask & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass
                        #elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        elif 0 < position_< len(order_book_ask)-1:    
                            #if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 
                            if order_book_ask[position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:                             
                                print 'Some error5(Ask & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                #print 'position and Best Price is True'  
                                pass
                        #elif position_ == len(order_book_ask[temp_ask])-1:
                        elif position_ == len(order_book_ask)-1:
                            #if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            if position_ > 0 and order_book_ask[position_-1:position_-1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                                print 'Some error5(Ask & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            elif position_ == 0:
                                #print 'position and Best Price is True'
                                pass                       
                            else:
                                #print 'position and Best Price is True'                        
                                pass
                        #elif position_ == 0 and len(order_book_ask[temp_ask]) == 1:
                        elif position_ == 0 and len(order_book_ask) == 1:
                            #print 'position and Best Price is True'
                            pass      
                    else:
                        #print 'before open market'
                        pass
                    #if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    if order_book_ask[(order_book_ask.OrderNumber == order_number_)&(order_book_ask.Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):                    
                        #order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                        order_book_ask = order_book_ask.drop(order_book_ask.index[[position_]]).reset_index(drop = True)
                    else:
                        #order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                        order_book_ask['QuantityDifference'][order_book_ask.OrderNumber == order_number_] = order_book_ask['QuantityDifference'][order_book_ask.OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]

        elif data.BidOrAsk[i] == 'B':
            data_bid_Quantity = data.BestQuantity[i]
            if int(data[['QuantityDifference']][i:i+1].values) > 0: 
                #if order_book_ask[temp_ask].Price[0] <= data[i:i+1].Price.iloc[0] and time_second < 32400:
                if order_book_ask.Price[0] <= data[i:i+1].Price.iloc[0] and time_second < 32400:
                    #for k in range(0,len(order_book_ask[temp_ask])):
                    for k in range(0,len(order_book_ask)):
                        #diff = order_book_ask[temp_ask].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                        diff = order_book_ask.QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                        #if order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                        if order_book_ask.Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                            #order_book_ask[temp_ask].QuantityDifference_[k] = diff
                            order_book_ask.QuantityDifference_[k] = diff
                            data[i:i+1].QuantityDifference_.iloc[0] = 0
                            break
                        #elif order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                        elif order_book_ask.Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                            #order_book_ask[temp_ask].QuantityDifference_[k] = 0
                            order_book_ask.QuantityDifference_[k] = 0
                            data[i:i+1].QuantityDifference_.iloc[0] = - diff
                            pass
                        else:
                            break
                if data.TimeStamp[i] == x2[temp_bid]:
                    #print 'timestamp = %s'%(data.TimeStamp[i])
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    #order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                    order_book_bid = insert(order_book_bid,data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                
                    #length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price >= data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                    if time_second > 32400 and time_second < 57300:
                        #if position_ == 0 and len(order_book_bid[temp_bid]) > 1:
                        if position_ == 0 and len(order_book_bid) > 1:
                            #if order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                
                            if order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                                            
                                print 'Some error1(Bid & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass     
                        #elif 0 < position_< len(order_book_bid[temp_bid])-1:
                        elif 0 < position_< len(order_book_bid)-1: 
                            #if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            if order_book_bid[position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                             
                                print 'Some error1(Bid & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break    
                            else:
                                #print 'position and Best Price is True'   
                                pass
                        #elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[temp_bid]) > 1:      
                        elif position_ == len(order_book_bid)-1 and len(order_book_bid) > 1:    
                            #if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            if order_book_bid[position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:                        
                                print 'Some error1(Bid & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'                
                                pass
                        elif position_ == 0 and len(order_book_bid[temp_bid]) == 1:
                            #print 'position and Best Price is True'
                            pass 
                    else:
                        #print 'before open market'
                        pass
                elif data.TimeStamp[i] != x2[temp_bid]:
                    #print temp_bid
                    #print 'timestamp = %s'%(data.TimeStamp[i])
                    if temp_bid == 0:
                        best_price = data[i:(i+1)]['BestPrice']             
                        position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                        temp_bid = temp_bid + 1
                        #order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                        #order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                 
                        order_book_bid = insert(order_book_bid,data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                                     
                        if time_second > 32400 and time_second < 57300:
                            #if position_ == 0  and len(order_book_bid[temp_bid]) > 1:
                            if position_ == 0  and len(order_book_bid) > 1:
                                #if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                if order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid['Price'][0:1].iloc[0] != data['BestPrice'][i]:                                
                                    print 'Some error2(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                    break
                                else:
                                    #print 'position and Best Price is True'                     
                                    pass       
                            #elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            elif 0 < position_< len(order_book_bid)-1:    
                                #if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                if order_book_bid[position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                                
                                    print 'Some error2(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                    break
                                else:
                                    #print 'position and Best Price is True'  
                                    pass
                            #elif position_ == len(order_book_bid[temp_bid])-1:
                            elif position_ == len(order_book_bid)-1:
                                #if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                if order_book_bid[position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid['Price'][0:1].iloc[0] != data['BestPrice'][i]:                                
                                    print 'Some error2(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])                                        
                                    pass
                                else:
                                    #print 'position and Best Price is True'
                                    pass
                            #elif position_ == 0 and len(order_book_bid[temp_bid]) == 1:
                            elif position_ == 0 and len(order_book_bid) == 1:        
                                    #print 'position and Best Price is True'
                                    pass
                        else:
                            #print 'before open market'
                            pass
                    else:
                        if round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) > 0.03 or\
                        round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) < 0:
                            #order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)                    
                            order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid,order_book_ask,data,i)                    
                            if data[i-1:i].BidOrAsk.iloc[0] == 'B':
                                if order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                    #print 'Best bid quantity is ture'
                                    pass
                                else:
                                    print 'Best bid quantity is false'
                                    break 
                            else:
                                j = i - 1
                                while j >= 1:
                                    #print j
                                    #print data[j-1:j].BidOrAsk.iloc[0]
                                    if data[j-1:j].BidOrAsk.iloc[0] == 'B':
                                        if order_book_bid_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                            #print 'Best bid quantity is ture'
                                            break
                                        else:
                                            print 'Best bid quantity is false'
                                            break
                                    else:
                                        j = j - 1
                                        pass
                        position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                        #order_book_bid.append(order_book_bid[temp_bid])
                        temp_bid = temp_bid + 1 
                        #order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)   
                        order_book_bid = insert(order_book_bid,data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                       
                        if time_second > 32400 and time_second < 57300:
                            #if position_ == 0 and len(order_book_bid[temp_bid]) > 1:
                            if position_ == 0 and len(order_book_bid) > 1:
                                #if order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                if order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid['Price'][0:1].iloc[0] != data['BestPrice'][i]:                                
                                    print 'Some error3(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                    break
                                else:
                                    #print 'position and Best Price is True'                     
                                    pass
                            #elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            elif 0 < position_< len(order_book_bid)-1:    
                                #if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                if order_book_bid[position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                                
                                    print 'Some error3(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                    break
                                else:
                                    #print 'position and Best Price is True'  
                                    pass
                            #elif position_ == len(order_book_bid[temp_bid])-1:
                            elif position_ == len(order_book_bid)-1:   
                                #if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                if order_book_bid[position_-1:position_-1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid['Price'][0:1].iloc[0] != data['BestPrice'][i]:                                
                                    print 'Some error3(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                    break
                                else:
                                    #print 'position and Best Price is True'                        
                                    pass
                            elif position_ == 0 and len(order_book_bid[0]) == 1:
                                #print 'position and Best Price is True'
                                pass
                        else:
                            #print 'before open market'
                            pass
            elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
                if data.TimeStamp[i] == x2[temp_bid]: 
                    #print 'timestamp = %s'%(data.TimeStamp[i])
                    order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                    #position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                    position_ = order_book_bid[order_book_bid.OrderNumber == order_number_].index[0]                
                    price_ = data[i:i+1]['Price'].iloc[0]
                    if time_second > 32400 and time_second < 57300:
                        #if position_ == 0 and len(order_book_bid[temp_bid]) > 1:
                        if position_ == 0 and len(order_book_bid) > 1:    
                            #if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            if order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                            
                                print 'Some error4(Bid & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass               
                        #elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        elif 0 < position_< len(order_book_bid)-1:      
                            #if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            if order_book_bid[position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                            
                                print 'Some error4(Bid & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'  
                                pass
                        #elif position_ == len(order_book_bid[temp_bid])-1:
                        elif position_ == len(order_book_bid)-1:    
                            #if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            if order_book_bid[position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:                            
                                print 'Some error4(Bid & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            elif position_ == 0:
                                #print 'position and Best Price is True'
                                pass
                            else:
                                #print 'position and Best Price is True'                        
                                pass
                        #elif position_ == 0 and len(order_book_bid[temp_bid]) == 1:
                        elif position_ == 0 and len(order_book_bid) == 1:
                            #print 'position and Best Price is True'
                            pass
                    else:
                        #print 'before open market'
                        pass
                    #if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    if order_book_bid[(order_book_bid.OrderNumber == order_number_)&(order_book_bid.Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):                    
                        #order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                        order_book_bid = order_book_bid.drop(order_book_bid.index[[position_]]).reset_index(drop = True)                
                    else:
                        #order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]                
                        order_book_bid['QuantityDifference'][order_book_bid.OrderNumber == order_number_] = order_book_bid['QuantityDifference'][order_book_bid.OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]                            
                elif data.TimeStamp[i] != x2[temp_bid]:
                    #print 'timestamp = %s'%(data.TimeStamp[i])
                    if round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) > 0.03 or\
                    round(float(data[i:i+1].TimeStamp.iloc[0][18:29]) - float(data[i-1:i].TimeStamp.iloc[0][18:28]),4) < 0:
                        #order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                        order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid,order_book_ask,data,i)
                        if data[i-1:i].BidOrAsk.iloc[0] == 'B':
                            if order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                #print 'Best bid quantity is ture'
                                pass
                            else:
                                print 'Best bid quantity is false'
                                break 
                        else:
                            j = i - 1
                            while j >= 1:
                                #print j
                                #print data[j-1:j].BidOrAsk.iloc[0]
                                if data[j-1:j].BidOrAsk.iloc[0] == 'B':
                                    if order_book_bid_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                        #print 'Best bid quantity is ture'
                                        break
                                    else:
                                        print 'Best bid quantity is false'
                                        break
                                else:
                                    j = j - 1
                                    pass 
                    #order_book_bid.append(order_book_bid[temp_bid])
                    order_number_ =  data['OrderNumber'][i:i+1].iloc[0]
                    #position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                    position_ = order_book_bid[order_book_bid.OrderNumber == order_number_].index[0]
                    price_ = data[i:i+1]['Price'].iloc[0]
                    temp_bid = temp_bid + 1
                    if time_second > 32400 and time_second < 57300:
                        #if position_ == 0 and len(order_book_bid[temp_bid]) > 1:
                        if position_ == 0 and len(order_book_bid) > 1:   
                            #if order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            if order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                            
                                print 'Some error5(Bid & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'                     
                                pass
                        #elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        elif 0 < position_< len(order_book_bid)-1:    
                            #if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            if order_book_bid[position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                            
                                print 'Some error5(Bid & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                #print 'position and Best Price is True'  
                                pass
                        #elif position_ == len(order_book_bid[temp_bid])-1: 
                        elif position_ == len(order_book_bid)-1:    
                            #if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            if order_book_bid[position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:                            
                                print 'Some error5(Bid & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            elif position_ == 0:
                                #print 'position and Best Price is True'
                                pass
                            else:
                                #print 'position and Best Price is True'                        
                                pass    
                        #elif position_ == 0 and len(order_book_bid[temp_bid]) == 1:
                        elif position_ == 0 and len(order_book_bid) == 1:
                            #print 'position and Best Price is True'
                            pass  
                    else:
                        #print 'before open market'
                        pass
                    #if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    if order_book_bid[(order_book_bid.OrderNumber == order_number_)&(order_book_bid.Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                        #order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                        order_book_bid = order_book_bid.drop(order_book_bid.index[[position_]]).reset_index(drop = True)                
                    else:
                        #order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                        order_book_bid['QuantityDifference'][order_book_bid.OrderNumber == order_number_] = order_book_bid['QuantityDifference'][order_book_bid.OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]


In [14]:
year = 2014
month = 1
path = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_'
best_price_number = 5

## 2014/01/02,03,06,07,08,09,10,13,14,15,16,17,20,21,23,24,27,28,29

In [20]:
day_ = [2,3,6,7,8,9,10]#,13,14,15,16,17,20,21,23,24,27,28,29]

In [ ]:
start = time.time()
for i in day_:
    order_book_tranform(year,month,i,path,best_price_number)
end = time.time()
print "Total time = %f"%(end - start)

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:64: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:65: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:411: SettingWithCop

In [19]:
pd.read_table('order_book_2014_1_2.csv',sep=',')#[50:120]

,Bid,Bid_Quantity,Ask,Ask_Quantity
0,TimeStamp,2014-01-02D04:19:51.857166800,NaN,NaN
1,700500,2,715500.0,2.0
2,699000,2,717500.0,1.0
3,697500,1,719500.0,1.0
4,690500,1,740000.0,2.0
5,690000,2,745000.0,10.0
6,TimeStamp,2014-01-02D08:45:00.131582600,NaN,NaN
7,682000.0,111.0,0.0,210.0
8,NaN,NaN,715500.0,2.0
9,NaN,NaN,717500.0,1.0


In [ ]:
pd.read_table('order_book_2014_1_3.csv',sep=',')

In [ ]:
pd.read_table('order_book_2014_1_6.csv',sep=',')

In [ ]:
pd.read_table('order_book_2014_1_7.csv',sep=',')

In [ ]:
pd.read_table('order_book_2014_1_8.csv',sep=',')

In [ ]:
pd.read_table('order_book_2014_1_10.csv',sep=',')